In [0]:
use catalog uc_prod

### Getting the distinct states from the staing master table (silver.daily_pricing_silver)

In [0]:
create or replace table silver.dim_state_staging_st1
select distinct state_name from silver.daily_pricing_silver
where record_updated_date > (
      select nvl(max(process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs 
      where process_name = 'reporting_dim_state_gold' and process_status ='completed'
                            )


### Add the distinct unique id for the state items

In [0]:
create or replace table uc_prod.silver.dim_state_gold_st_2
select s.state_name, 
case when t.state_id is null 
  then row_number() over(order by s.state_name) 
  else t.state_id end as state_ID_st2
,t.state_id as gold_state_id
from uc_prod.silver.dim_state_staging_st1 s
left join gold.reporting_dim_state_gold t
on t.state_name = s.state_name
where t.state_name is null or t.state_id is not null

In [0]:
create or replace table uc_prod.silver.dim_state_gold_st_3
select 
state_name,
case when  src.gold_state_id is null
then (src.state_ID_st2+max_id) 
else  src.gold_state_id end as STATE_ID,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date 
from uc_prod.silver.dim_state_gold_st_2 as src
cross join (select nvl(max(STATE_ID),0) as max_id from uc_prod.gold.reporting_dim_state_gold)

In [0]:
merge into gold.reporting_dim_state_gold as tgt
using uc_prod.silver.dim_state_gold_st_3 as src
on tgt.STATE_ID = src.STATE_ID
when matched then update 
  set tgt.lakehouse_updated_date = current_timestamp()
when not matched then  
  insert (State_ID,STATE_NAME,lakehouse_inserted_date,lakehouse_updated_date)
  values
  (src.STATE_ID,src.STATE_NAME,src.lakehouse_inserted_date,src.lakehouse_updated_date)

### update the water mark table

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_dim_state_gold',
current_timestamp(),
'completed'
)